Escolhemos este dataset afim de possivelmente criar um pequeno sistema de recomendação baseado em regras de associacao, O dataset contem inicialmente 4 colunas:
userId - Id do usuario repsonsavel por uma nota
movieId  - Id do filme avaliado pelo usuario
rating - nora dada pelo usuario a um determinado filme
timestamp - o

In [145]:
import pandas as pd

In [146]:
df_movies = pd.read_csv('data/ratings.csv')

In [147]:
#Exemplo de dados
df_movies.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [148]:
#A primeira coisa a fazer sera criar uma pivot table
#Tornando o user id como linha, o movie id como coluna e o ratings como valor
#tornando o dataset ocm a cara transacional
df_movies = pd.pivot_table(df_movies, values='rating', index=['userId'],columns=['movieId'], aggfunc=np.max)

In [149]:
#Nosso dataset e muito grande e esparso, depois devarios testes
#prefirimos filtrar apenas os filmes que receberam maiores ratings (para melhorar as regras)
#Um dataset muito esparso impacta demais os valores de threshold de suporte
df_counter_ratings = pd.DataFrame(df_movies.sum(axis=0), columns=['sum_ratings'])
top_movies = list(df_counter.sort_values(by='sum_ratings', ascending=False)[:50].index)

In [150]:
#Novo head dos dados
df_top_movies = df_movies[top_movies]
df_top_movies.head()

movieId,318,356,296,593,260,2571,527,480,1196,1,...,377,3578,1265,1240,1193,1136,1097,648,1221,1291
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3.0,4.0,3.0,NaN,NaN,4.0,4.0,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,5.0,4.5,3.0,NaN,NaN,3.0,NaN,NaN,NaN,...,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5.0,5.0,NaN,5.0,NaN,NaN,5.0,5.0,NaN,...,NaN,NaN,5.0,5.0,NaN,5.0,5.0,NaN,NaN,5.0
5,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,2.5,NaN


In [151]:
#Iremos transformar em True usuarios que dram ratings maior que 3
#Tomando como verdade que eles 'indicariam' o filme
df_top_movies = df_top_movies.applymap(lambda k: k>=3)

In [152]:
#Alem do movimento acima, vamos tambem filtrar usuarios que viram pelo menos
#10 filmes, para também melhorar as regras ao final
df_counter_views =  pd.DataFrame(df_top_movies.sum(axis=1), columns=['number_views'])
top_viewers = list(df_counter_views.loc[df_counter_views['number_views'] > 10].index.values)

In [155]:
#nosso novo dataset contem 50 colunas e 373 linhas
df_top_movies = df_top_movies.loc[top_viewers]
print(df_top_movies.shape)
df_top_movies.head()

(373, 50)


movieId,318,356,296,593,260,2571,527,480,1196,1,...,377,3578,1265,1240,1193,1136,1097,648,1221,1291
userId,,,,,,,,,,,,,,,,,,,,,
2,False,True,True,True,False,False,True,True,False,False,...,True,False,False,False,False,False,False,False,False,False
3,True,True,True,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,True,False,True,False,False,True,True,False,...,False,False,True,True,False,True,True,False,False,True
7,True,True,False,False,True,False,False,True,True,True,...,True,False,False,True,False,True,True,False,False,True
8,True,True,True,True,True,True,True,False,True,False,...,False,True,True,False,False,False,False,False,False,True


In [158]:
#Por fim, vamos tranformar o dataset no mesmo formato do
#dataset originalmente trabalhdo (Congressional_Voting_Records) para o algoritmo rodar sem problemas
df_top_movies = df_top_movies.replace(True,'y')
df_top_movies = df_top_movies.replace(False,'n')


In [159]:
#Salvamos para csv estamos prontos para rodar o algoritmo
df_top_movies.to_csv("data/movies_ratings.csv",index=False)